In [4]:
from __future__ import division
from sympy import *
a, b, c, d, e, f = symbols('a b c d e f')
init_printing()
init_printing(use_latex='mathjax', latex_mode='equation')


import pyperclip
def lx(expr):
    pyperclip.copy(latex(expr))
    print(expr)

import numpy as np
import matplotlib as plt

# Problem 4.2

## Trivial Example

Consider the list:

```python
easyData = [[a], [b], [c], [d]]
```

In [ ]:
A common need is to flatten that list, the simplest means by which to do that is with *list comprehension* like so:

In [5]:
easyData = [[a], [b], [c], [d]]
[item for sublist in easyData for item in sublist]

[a, b, c, d]

In [ ]:
This technique is equivalent to using nested `for` loops but it is significantly quicker. [^551] 

It does however require that the level of nesting to be consistent across elements, which is not always the case, consider this example:

[^551]: [python - Flatten list of lists - Stack Overflow](https://stackoverflow.com/a/11264751/12843551)


## Complex Example

```python
data = [[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]
```

Functionality such as this isn't usually built into *Python*, instead the idea is to build it yourself.

The `functools` library does however offer some tooling such as `reduce` that may assist in this, however in this case it is arguably simpler to use the built in functions.

Initially iterate a loop over the list to print the values:

In [6]:
data = [[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]

for item in data:
    if type(item) is list:
        for subitem in item:
            print(subitem)
    else:
        print(item)



[[a, b], 2]
[[c, d], 3]
[[a, b], 2]
[[e, f], 3]


This appears to successfully reduce the level of the list by one, so now instead of printing lets append the values onto a new list:

In [11]:


new_list = []
for item in data:
    if type(item) is list:
        for subitem in item:
            new_list.append(subitem)
    else:
        new_list.append(item)

print(str(new_list) + '\n' + str(data))



[[[a, b], 2], [[c, d], 3], [[a, b], 2], [[e, f], 3]]
[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]


In [ ]:
Again it appears that this has worked and so now we would want to wrap this into a function so it can be a part of our toolbox:

In [16]:
def flat(list):
    new_list = []
    for item in list:
        if type(item) is list:
            for subitem in item:
                new_list.append(subitem)
        elif type(item) is not list:
            new_list.append(item)
        else:
            print("ERROR: Element is neither a list nor a non-list")
            return 0
    return new_list

print(str(flat(data)) + '\n' + str(data))



[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]
[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]


Notice that when wrapped in a function this method **does not work**.

This can be very confusing and the reason that it fails is because the expression `type(item) is list` statement evaluates as false when inside a loop inside a function.

I can't quite explain why this behaviour occurs, but I can say that many people have had to deal with testing whether or not a *Python* object is iterable [^128].

[^128]: 
[In Python, how do I determine if an object is iterable? - Stack Overflow](https://stackoverflow.com/a/61139278/12843551)

The function `iter()` is a function that will *get an iterator from an object*, however more importantly it will throw an error if an item cannot be pulled out of an object, this behaviour can be used with a `try`/`catch` statement to build a logical test for an object that has items that can be removed [^1282]

[^1282]: For an object that is iterable but does not have items that can be returned the `collections.Iterable()` function may be helpful.


In [17]:
# This is good but notice that it fails:

def isIterable(x):
    try:
        iter(x)
        return True
    except:
        return False

isIterable([1,2])


True

In [ ]:
This Try/Catch Function returns the following output given input, which is exactly what is desired in this situation.

| ***Argument*** | ***Output*** |
| ---            | ---          |
| `[1,2]`        | `True`       |
| `[3]`          | `False`      |
| `4`            | `False`      |


Now replacing `type(list) is list` with our `isIterable(list)` function we can see that it does indeed flatten the list:

In [19]:

def flat(list):
    new_list = []
    for item in list:
        if isIterable(item):
            for subitem in item:
                new_list.append(subitem)
        elif not isIterable(item):
            new_list.append(item)
        else:
            print("ERROR: Element is neither a iterable or a non-iterable")
            return 1
    return new_list

print(str(data) + '\n' + str(flat(flat(data))))


[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]
[[a, b], 2, [c, d], 3, [a, b], 2, [e, f], 3]


In [ ]:
This is also tollerant to larger arguments, for example [^wn]:

[^wn]: (silent failure may be undesirable)

In [20]:
flat(flat(flat(data)))
flat(flat(flat(flat(data))))




[a, b, 2, c, d, 3, a, b, 2, e, f, 3]

Now to give this an argument to control the depth of flattening recursion or a `for` loop can be used.

Generally any problem that can be solved via recursion can be solved via a for loop and vice versa, however in languages such as *Python* and ***R*** `for` loops **may** perform better due to the overheads of calling a function. [^pf]

In this situation a `for` loop is easier to comprehend:

[^pf]: Test this if performance critical

Putting this into practice we get:

In [23]:
def flatten(list, n):
    for i in range(n-1):
        list = flat(list)
    return list

data
print(str(data) + '\n' + str(flatten(data, 4)))

[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]
[a, b, 2, c, d, 3, a, b, 2, e, f, 3]


In [ ]:
This shows that it does indeed work.

## Solution

For convenience sake the entire solution is contained below:



In [27]:
def flatten(list, n):
    for i in range(n-1):
        list = flat(list)
    return list

def flat(list):
    new_list = []
    for item in list:
        if isIterable(item):
            for subitem in item:
                new_list.append(subitem)
        elif not isIterable(item):
            new_list.append(item)
    return new_list


def isIterable(x):
    try:
        iter(x)
        return True
    except:
        return False


In [26]:

data
print(str(data) + '\n' + str(flatten(data, 4)))


[[[[a, b], 2], [[c, d], 3]], [[[a, b], 2], [[e, f], 3]]]
[a, b, 2, c, d, 3, a, b, 2, e, f, 3]
